In [372]:
import sys
print(sys.version)

3.12.11 (main, Jun  3 2025, 15:41:47) [Clang 17.0.0 (clang-1700.0.13.3)]


In [373]:
%load_ext autoreload
%autoreload 0

import sys
import os
import json
from pprint import pprint
from operator import itemgetter

from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
import math
from datetime import date, datetime, timedelta
from scipy import stats
from nfl_teams import NFL_TEAM_MAP
from odds import get_metabet_spread, get_current_rankings, get_fantasy_def_points_against
from collections import defaultdict

def get_most_recently_created_file_with_extension(folder, extension):
    files = [f for f in os.listdir(folder) if f.endswith(extension)]
    return max(files, key=lambda x: os.path.getctime(f"{folder}/{x}"))

# https://www.pro-football-reference.com/years/2021/fantasy.htm
DATA_FOLDER = './data25'
ACTIVE_FOLDER = './active'
UPLOAD_FOLDER = './upload'
SALARY_FILE = f"{DATA_FOLDER}/{get_most_recently_created_file_with_extension(DATA_FOLDER, 'csv')}"
SEASON_START = '09/01/2025'

def get_week_relative_to_start(season_start=SEASON_START):
    start = datetime.strptime(season_start, '%m/%d/%Y')
    today = datetime.today()
    num_days = (today - start).days + 1
    return math.ceil(num_days / 7)


WEEK = max(get_week_relative_to_start(), 1)

WEIGHTED = True

ACTIVE_FILE = f"{ACTIVE_FOLDER}/data.csv"
UPLOAD_FILE = f"{UPLOAD_FOLDER}/upload.csv"
MAX_PLAYED = WEEK - 1

df = pd.read_csv(SALARY_FILE, na_values= '')

print('ready', SALARY_FILE, WEEK)
# print(df.head())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready ./data25/FanDuel-NFL-2025 EST-12 EST-28 EST-124752-players-list (4).csv 17


In [374]:
# Check df immediately after loading
print(f"\nDataFrame loaded with {df.shape[0]} rows and {df.shape[1]} columns")
print(f"Positions: {df['Position'].unique()}")
print(f"Position value counts:\n{df['Position'].value_counts()}")



DataFrame loaded with 562 rows and 17 columns
Positions: ['WR' 'RB' 'QB' 'TE' 'D']
Position value counts:
Position
WR    217
TE    127
RB    121
QB     75
D      22
Name: count, dtype: int64


In [375]:
# Manual step: Load NFL team rankings from a local CSV or DataFrame instead of get_current_rankings.
# If you do not have a rankings file, this cell will safely skip and the rest of the notebook will work without rankings.
import os
import pandas as pd
RANKINGS_CSV = './ranking/defense_1.json'  # Update this path to your rankings file (CSV or JSON)
if os.path.isfile(RANKINGS_CSV):
    try:
        if RANKINGS_CSV.endswith('.csv'):
            ranking_df = pd.read_csv(RANKINGS_CSV)
        elif RANKINGS_CSV.endswith('.json'):
            ranking_df = pd.read_json(RANKINGS_CSV)
        else:
            raise ValueError('Unsupported file format for rankings.')
    except Exception as e:
        print('Error loading rankings:', e)
        ranking_df = pd.DataFrame()  # Empty fallback
else:
    print(f'Rankings file not found: {RANKINGS_CSV}. Proceeding without rankings.')
    ranking_df = pd.DataFrame()  # Empty fallback
    # You can also paste your DataFrame here manually if needed.

def get_abbr(x):
    try:
        short_team = NFL_TEAM_MAP[x]
        print(x, short_team)
        return short_team
    except Exception as e:
        print(e, x)

if not ranking_df.empty and 'team_fk__full_name' in ranking_df.columns:
    ranking_df['team'] = ranking_df['team_fk__full_name'].apply(get_abbr)
else:
    print('Ranking DataFrame is empty or missing expected columns.')
# DEF_KEYS = ['passing_interceptions_rank'
# ranking_df['def_rank'] = ranking_df.apply(lambda row: np.avg([row[k] for k in DEF_KEYS]))
try:
    if 'df' in globals() and 'Team' in df.columns:
        num_teams = df['Team'].nunique()
    else:
        num_teams = 0
except Exception as e:
    print('Error determining number of teams from data CSV:', e)
    num_teams = 0

ranking_df.shape
print(ranking_df.columns.values)

Rankings file not found: ./ranking/defense_1.json. Proceeding without rankings.
Ranking DataFrame is empty or missing expected columns.
[]


In [376]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [377]:
# rankings = ranking_df.to_dict('records') if not ranking_df.empty else []
if ranking_df is not None and not ranking_df.empty and 'team' in ranking_df.columns:
    rankings = ranking_df.to_dict('records')
    print(rankings[0]['team'])
    rankings = {x['team']: x for x in  rankings}
    teams = set(rankings.keys())
    print(len(teams), teams)
else:
    print('No rankings data loaded. Proceeding without rankings.')
    rankings = {}
    teams = set()

No rankings data loaded. Proceeding without rankings.


In [378]:
spread_df = get_metabet_spread(WEEK)
favor_map = {}
z_map = {}
z_scores = {}
print(spread_df)

return cached data week 17
    Unnamed: 0 AwayTeam HomeTeam  PointSpread  OverUnder
0            0      DAL      WAS         9.12      50.23
1            1      DET      MIN         7.44      44.91
2            2      DEN       KC        13.52      37.59
3            3      HOU      LAC        -1.53      39.50
4            4      BAL       GB        -4.50      40.39
5            5       TB      MIA         5.86      44.46
6            6       NE      NYJ        13.50      42.85
7            7      PIT      CLE         3.00      33.97
8            8      ARI      CIN        -7.04      52.76
9            9       NO      TEN         2.50      39.44
10          10      JAC      IND         6.25      48.12
11          11      SEA      CAR         7.09      42.50
12          12      NYG       LV         1.07      41.46
13          13      PHI      BUF        -1.46      44.02
14          14      CHI       SF        -3.02      52.13
15          15      LAR      ATL         8.00      49.65


In [379]:


if 'OverUnder' in spread_df.columns.values:
    points = list(spread_df['OverUnder'])
    zs = stats.zscore(points)
    for i, p in enumerate(points):
        z_scores[p] = 0 if np.isnan(zs[i]) else zs[i]

# Vegas odds convention: PointSpread is from the home team's perspective
# Negative spread = home team is favored by that amount
# Positive spread = away team is favored by that amount
# For favor_map: positive = team is unfavored, negative = team is favored
for index, row in spread_df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    # PointSpread represents home team's advantage
    # Direct mapping: home team gets the spread as-is, away team gets negated
    favor_map[home] = row['PointSpread']   # Home team (negative = favored, positive = unfavored)
    favor_map[away] = -row['PointSpread']  # Away team (opposite of home)
    if 'OverUnder' in row:
        z_map[home] = z_scores[row['OverUnder']]
        z_map[away] = z_scores[row['OverUnder']]



if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    z_map['JAC'] = z_map['JAX']

if 'LVS' in favor_map:
    favor_map['LV'] = favor_map['LVS']
    z_map['LV'] = z_map['LVS']

sort_key = itemgetter(1)
sorted(favor_map.items(), key=sort_key)

print(spread_df)
z_scores

    Unnamed: 0 AwayTeam HomeTeam  PointSpread  OverUnder
0            0      DAL      WAS         9.12      50.23
1            1      DET      MIN         7.44      44.91
2            2      DEN       KC        13.52      37.59
3            3      HOU      LAC        -1.53      39.50
4            4      BAL       GB        -4.50      40.39
5            5       TB      MIA         5.86      44.46
6            6       NE      NYJ        13.50      42.85
7            7      PIT      CLE         3.00      33.97
8            8      ARI      CIN        -7.04      52.76
9            9       NO      TEN         2.50      39.44
10          10      JAC      IND         6.25      48.12
11          11      SEA      CAR         7.09      42.50
12          12      NYG       LV         1.07      41.46
13          13      PHI      BUF        -1.46      44.02
14          14      CHI       SF        -3.02      52.13
15          15      LAR      ATL         8.00      49.65


{50.23: 1.1918156811973144,
 44.91: 0.1742895951038793,
 37.59: -1.2257650947840788,
 39.5: -0.8604502781603086,
 40.39: -0.6902250494717451,
 44.46: 0.08822065925011213,
 42.85: -0.2197148668044799,
 33.97: -1.9181418676521689,
 52.76: 1.6757143649973878,
 39.44: -0.8719261362741447,
 48.12: 0.7882480041940911,
 42.5: -0.28665737246852196,
 41.46: -0.48557224644167446,
 44.02: 0.004064366415317202,
 52.13: 1.5552178548021136,
 49.65: 1.0808823860969026}

In [380]:
# Debug: START OF FILTERING
print("\n=== START OF FILTERING CELL ===")
print(f"df shape at start: {df.shape}")
print(f"Positions in df: {df['Position'].unique()}")
print(f"Position counts at start:\n{df['Position'].value_counts()}")

set_teams = set(df['Team'])
# Automatically set SINGLE_GAME based on number of unique teams in the data
if len(set_teams) == 2:
    SINGLE_GAME = True
else:
    SINGLE_GAME = False

MIN_SALARY = 1600 if SINGLE_GAME else 4900
print('SINGLE_GAME', SINGLE_GAME, len(set_teams), set_teams, MIN_SALARY)
# df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

# Players with injury status.
# for index, p in df[~df['Injury Indicator'].isna()].iterrows():
#     print(p['Injury Indicator'], p['Name'], p['Team'])

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 1
half_teams = int(len(ordered_teams)/2)
# For single-game contests, include all teams IN THE DATA. Otherwise, select top favored teams.
if SINGLE_GAME:
    top_teams = list(set_teams)  # Include only teams that are in the player data
else:
    top_teams = ordered_teams[team_offset:team_offset+int(half_teams)]  # Select middle to top (favored) teams
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = set([*questionable_players, *low_salary_players])

# Let the optimizer decide which defenses to use based on value/merit
# The adjusted projections already incorporate team quality via favor_map weighting

readd = ['Justin Herbert', 'Patrick Taylor Jr.', 'Bailey Zappe', "D'Andre Swift", 'David Montgomery',]
for p in readd:
    if p in excluded_players:
        excluded_players.remove(p)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")

questionable_df = df[(df['Name'].isin(set(questionable_players)))]

excluded_df = df[(df['Name'].isin(excluded_players))]

df = df[(~df['Name'].isin(excluded_players))]# & (df['FPPG'] > 0) & (df['Played'] > 0)




=== START OF FILTERING CELL ===
df shape at start: (562, 17)
Positions in df: ['WR' 'RB' 'QB' 'TE' 'D']
Position counts at start:
Position
WR    217
TE    127
RB    121
QB     75
D      22
Name: count, dtype: int64
SINGLE_GAME False 22 {'SEA', 'NYJ', 'MIA', 'SF', 'NYG', 'CHI', 'NO', 'LAR', 'ARI', 'LV', 'PIT', 'IND', 'JAC', 'CAR', 'ATL', 'NE', 'TB', 'TEN', 'CIN', 'BUF', 'CLE', 'PHI'} 4900
             FPPG      Played        Salary  Tier  Unnamed: 14  Unnamed: 15
count  474.000000  562.000000    562.000000   0.0          0.0          0.0
mean     5.047988    7.405694   4921.352313   NaN          NaN          NaN
std      5.182554    5.602314   1238.928705   NaN          NaN          NaN
min     -1.000000    0.000000   3000.000000   NaN          NaN          NaN
25%      1.004167    2.000000   4000.000000   NaN          NaN          NaN
50%      3.485000    7.000000   4400.000000   NaN          NaN          NaN
75%      7.412500   13.000000   5700.000000   NaN          NaN          NaN


In [381]:
# Debug: Check df after filtering
print("\n=== AFTER FILTERING ===")
print(f"df shape: {df.shape}")
print(f"Positions in df: {df['Position'].unique()}")
print(f"Position counts:\n{df['Position'].value_counts()}")
print(f"\nExcluded players: {len(excluded_players)}")
print(f"Sample excluded players: {list(excluded_players)[:10]}")



=== AFTER FILTERING ===
df shape: (201, 19)
Positions in df: ['WR' 'RB' 'QB' 'TE' 'D']
Position counts:
Position
QB    66
WR    50
RB    44
D     22
TE    19
Name: count, dtype: int64

Excluded players: 361
Sample excluded players: ['Kaleb Johnson', 'Cameron Latu', 'Deion Hankins', 'Brycen Tremayne', 'Devin Duvernay', 'Daniel Brunskill', 'Elijah Arroyo', 'Lan Larison', 'Johnny Mundt', 'Reggie Gilliam']


In [382]:
# print all in df
# for index, p in enumerate(low_salary_players):
#     print(p)


In [383]:
REPLACE_MAP = {
    'LA': 'Los Angeles',
    '.':'',
}

def name_map(x):
    result = ' '.join(x.split(', ')[::-1])
    for k in REPLACE_MAP:
        result = result.replace(k, REPLACE_MAP[k])
    return result

In [384]:
print(winning_teams)

['DAL', 'DET', 'DEN', 'LAC', 'GB', 'TB', 'NE', 'PIT', 'CIN', 'NO', 'JAC', 'SEA', 'NYG', 'BUF', 'SF', 'LAR']


In [385]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-6# take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1) if week_number != 18 and os.path.isfile(f"./history/week{week_number}.csv")]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names]
print(f"Using {len(history_dfs)} weeks of history")
historic_averages = {}
if history_dfs:
    historic_data=pd.concat(history_dfs)



    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)

    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
    print(len(historic_averages))



[] 0
Using 0 weeks of history


In [386]:
excluded_bonus = defaultdict(lambda: 0)
injured_qb = defaultdict(lambda: False)

INJURY_FACTOR = .12

for index, p in questionable_df.iterrows():
    pos = p['Position']
    if pos in ['TE', 'WR', 'RB', 'QB']:
        points = p['FPPG']
        if points > 7.5 and p['Played'] >= WEEK / 2:
            injury_offset = min(points * INJURY_FACTOR, INJURY_FACTOR*10)
            if pos == 'QB':
                # subtract for QB
                amt = -injury_offset*2
                injured_qb[p['Team']] = True
            elif pos in ('RB', 'WR', 'TE'):
                amt = injury_offset*1.2
            else:
                amt = injury_offset

            print('bonus injured', p['Team'], p['Name'], amt)

            excluded_bonus[p['Team']] += amt

print(excluded_bonus)

bonus injured NO Chris Olave 1.44
bonus injured LAR Davante Adams 1.44
bonus injured ATL Drake London 1.44
bonus injured LV Brock Bowers 1.44
bonus injured CHI Rome Odunze 1.44
bonus injured SF George Kittle 1.44
bonus injured CLE Harold Fannin Jr. 1.3766399414062498
bonus injured ARI Marvin Harrison Jr. 1.4046545854048293
bonus injured IND Daniel Jones -2.4
bonus injured TEN Will Levis -2.4
bonus injured NYJ Justin Fields -2.4
bonus injured ATL Michael Penix Jr. -2.4
bonus injured NE Kayshon Boutte 1.1874461200420672
bonus injured NO Alvin Kamara 1.1022545055042612
bonus injured BUF Dalton Kincaid 1.3915636763139203
bonus injured MIA Alexander Mattison 1.1462400146484375
bonus injured CLE Quinshon Judkins 1.44
defaultdict(<function <lambda> at 0x1375d4fe0>, {'NO': 2.5422545055042614, 'LAR': 1.44, 'ATL': -0.96, 'LV': 1.44, 'CHI': 1.44, 'SF': 1.44, 'CLE': 2.8166399414062497, 'ARI': 1.4046545854048293, 'IND': -2.4, 'TEN': -2.4, 'NYJ': -2.4, 'NE': 1.1874461200420672, 'BUF': 1.391563676313

In [387]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./active/data.csv, (201, 19)


In [388]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
all_positions = set(df['Position'])
print(all_positions)


def get_nfl_positions():
    if SINGLE_GAME:
        positions = [[p, 0, 5] for p in all_positions]
        positions.append(['MVP', 1, 1])  # Exactly 1 MVP required
        return positions

    return [
        ['QB', 1, 1],
        ['RB', 2, 3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]

print(get_nfl_positions())

{'WR', 'D', 'QB', 'RB', 'TE'}
[['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]]


In [389]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']

In [390]:
# Debug: Check what's in df before groupby
print("DataFrame shape:", df.shape)
print("Positions in df:", df['Position'].unique())
print("Number of each position:", df['Position'].value_counts())
print("\nFirst few rows:")


DataFrame shape: (201, 19)
Positions in df: ['WR' 'RB' 'QB' 'TE' 'D']
Number of each position: Position
QB    66
WR    50
RB    44
D     22
TE    19
Name: count, dtype: int64

First few rows:


In [391]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)
max_salary = 9400
print(max_salary)

9400


In [392]:
qbs = df[df['Position'] == 'QB']
print(qbs[:10])

               Id Position First Name          Nickname Last Name       FPPG  \
8    124752-62239       QB       Josh        Josh Allen     Allen  24.029333   
9    124752-63336       QB        Joe        Joe Burrow    Burrow  16.653333   
12   124752-69531       QB      Jalen       Jalen Hurts     Hurts  19.944000   
13    124752-6654       QB    Matthew  Matthew Stafford  Stafford  21.863999   
14  124752-152316       QB      Drake        Drake Maye      Maye  20.952000   
21   124752-89951       QB     Trevor   Trevor Lawrence  Lawrence  20.384001   
23   124752-41535       QB      Baker    Baker Mayfield  Mayfield  16.817333   
25  124752-152113       QB      Caleb    Caleb Williams  Williams  18.906667   
26   124752-30231       QB     Jacoby   Jacoby Brissett  Brissett  17.861666   
30   124752-63589       QB        Sam       Sam Darnold   Darnold  15.968000   

    Played  Salary     Game Team Opponent Injury Indicator Injury Details  \
8       15    8800  PHI@BUF  BUF      PHI 

In [393]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000

ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX', 'K', 'MVP'] if SINGLE_GAME else ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX', 'K']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - (200 if SINGLE_GAME else 100)

if not SINGLE_GAME:
  ACTIVE_RULE_SET.max_players_per_team = 9

ACTIVE_RULE_SET.roster_size = 9 if not SINGLE_GAME else 6
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 59900, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX', 'K'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None, 'min_matchups': None, 'custom_rules': None}


In [394]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

allowed_map = get_fantasy_def_points_against(WEEK)

if 'Washington Football Team' in allowed_map:
    allowed_map['Washington Commanders'] = allowed_map['Washington Football Team']
allowed_map

return cached data week 17


{}

In [395]:
print(z_map)

{'WAS': 1.1918156811973144, 'DAL': 1.1918156811973144, 'MIN': 0.1742895951038793, 'DET': 0.1742895951038793, 'KC': -1.2257650947840788, 'DEN': -1.2257650947840788, 'LAC': -0.8604502781603086, 'HOU': -0.8604502781603086, 'GB': -0.6902250494717451, 'BAL': -0.6902250494717451, 'MIA': 0.08822065925011213, 'TB': 0.08822065925011213, 'NYJ': -0.2197148668044799, 'NE': -0.2197148668044799, 'CLE': -1.9181418676521689, 'PIT': -1.9181418676521689, 'CIN': 1.6757143649973878, 'ARI': 1.6757143649973878, 'TEN': -0.8719261362741447, 'NO': -0.8719261362741447, 'IND': 0.7882480041940911, 'JAC': 0.7882480041940911, 'CAR': -0.28665737246852196, 'SEA': -0.28665737246852196, 'LV': -0.48557224644167446, 'NYG': -0.48557224644167446, 'BUF': 0.004064366415317202, 'PHI': 0.004064366415317202, 'SF': 1.5552178548021136, 'CHI': 1.5552178548021136, 'ATL': 1.0808823860969026, 'LAR': 1.0808823860969026}


In [396]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 4
AVERAGE_WEIGHT = .5
MIN_PLAYED = min(int(WEEK * 0.6), 0)

defenses = []
qbs = []
mvps = []  # Track MVP players separately
mvp_base_names = set()  # Track base names of MVP players

MIN_QB_SALARY = 1000 if SINGLE_GAME else 6400

MIN_SCORE = 7
MAX_SCORE = 27
INJURED_QB_BONUS = 1.25
HOME_BONUS = .3

historic_data_used = 0

def build_starter_map(players, questionable_df):
    """
    Build a map of (team, position) -> (starter_name, starter_fppg, is_injured).
    
    Only tracks positions: QB, RB, WR, TE
    Identifies starters as the highest FPPG player for each position/team combo.
    Checks if starters are injured by looking them up in questionable_df.
    
    Args:
        players: List of Player objects
        questionable_df: DataFrame of players with injury indicators
        MIN_PLAYED: Minimum games played to be considered a starter
    
    Returns:
        dict: Mapping (team, pos) -> (starter_name, starter_fppg, is_injured)
    """
    starter_map = {}
    for p in players:
        if p.pos in ['QB', 'RB', 'WR', 'TE']:
            key = (p.team, p.pos)
            base_name = p.name.replace(' (MVP)', '')
            current_fppg = float(p.kv_store.get('FPPG') or 0)
            
            # Check if this player is injured
            is_injured = base_name in [q['Name'] for _, q in questionable_df.iterrows()] if not questionable_df.empty else False
            
            # Only store if this is the highest FPPG for this position/team (starter)
            if key not in starter_map or current_fppg > starter_map[key][1]:
                starter_map[key] = (base_name, current_fppg, is_injured)
    
    return starter_map


def filter_mvps(mvps, players, starter_map):
    """
    Filter MVP players based on games played and starter injury status.
    
    Includes a player if:
    - Player has played >= MIN_PLAYED games (is a starter), OR
    - Player is a backup AND their position's starter is injured
    
    Args:
        mvps: List of tuples (name, proj, cost, value, pos, base_fppg, opponent)
        players: List of Player objects (to look up games played and team info)
        starter_map: Dict mapping (team, pos) -> (starter_name, starter_fppg, is_injured)
        MIN_PLAYED: Minimum games played threshold for starters
    
    Returns:
        list: Filtered MVP tuples with added fields (player_games, opp, status)
              Format: (name, proj, cost, value, pos, base_fppg, player_games, opp, status)
    """
    filtered_mvps = []
    
    for name, proj, cost, value, pos, base_fppg, opponent in sorted(mvps, key=lambda x: x[3], reverse=True):
        # Extract the base player name (remove " (MVP)" suffix)
        base_name = name.replace(' (MVP)', '')
        
        # Find the player in the original players list to check games played and team
        player_games = 0
        player_team = None
        player_pos = None
        for p in players:
            if p.name == base_name:
                player_games = int(float(p.kv_store.get('Played', 0)))
                player_team = p.team
                player_pos = p.pos
                break
        
        status = "Starter"
        is_backup = player_games < MIN_PLAYED
        
        # Check if this is a backup but their starter is injured
        starter_injured = False
        if is_backup and player_team and player_pos in ['QB', 'RB', 'WR', 'TE']:
            key = (player_team, player_pos)
            if key in starter_map:
                starter_name, starter_fppg, is_injured = starter_map[key]
                if is_injured:
                    starter_injured = True
                    status = f"(Starter {starter_name} injured)"
        
        # Include if starter, or if backup but starter is injured
        if player_games >= MIN_PLAYED or starter_injured:
            filtered_mvps.append((name, proj, cost, value, pos, base_fppg, player_games, opponent, status))
    
    return filtered_mvps

def calculate_home_bonus(p):
    """Calculate home/away bonus."""
    teams = p.matchup.split('@')
    is_home = p.team == teams[1]
    return HOME_BONUS if is_home else -HOME_BONUS

def calculate_overunder_bonus(p, point_bonus):
    """Calculate over/under bonus based on position.
    For defenses, heavily weight the over/under since they score better in low-scoring games.
    """
    if not point_bonus:
        return 0
    if p.pos in ['D', 'MVP']:
        # For defenses: heavily penalize high O/U games, heavily reward low O/U games
        # Multiply by larger factor (3x vs 1.5x for other positions)
        return -point_bonus * 3.0
    return point_bonus * 1.5

def calculate_ranking_bonus(p, opponent):
    """Calculate ranking-based bonus."""
    current_rank = rankings.get(p.team, {}).get('points_rank_def', 0) if 'rankings' in globals() else 0
    opp_rank = (num_teams - rankings.get(opponent, {}).get('offensive_yards_rank', 0)) if 'rankings' in globals() else 0
    overall_diff = opp_rank - current_rank
    return overall_diff / num_teams

def calculate_injury_bonuses(p, opponent):
    """Calculate all injury-related bonuses."""
    bonuses = 0

    # Opponent QB injury bonus
    if injured_qb.get(opponent, False):
        qb_bonus = INJURED_QB_BONUS if p.pos in ['D', 'MVP'] else INJURED_QB_BONUS / 2
        # print(f"hurt {opponent} QB, {qb_bonus} bonus to {p.pos} {p.team} {p.name}")
        bonuses += qb_bonus

    # Team injury bonus
    if p.pos == 'QB':
        bonuses += -excluded_bonus.get(p.team, 0)
    elif p.pos in ['D', 'MVP']:
        bonuses += excluded_bonus.get(p.team, 0) / 2
    else:
        bonuses += excluded_bonus.get(p.team, 0)

    # RB bonus for injured QB on own team
    if p.pos == 'RB' and injured_qb.get(p.team, False):
        # print(f"hurt {p.team} QB, {INJURED_QB_BONUS} bonus to {p.pos} {p.name}")
        bonuses += INJURED_QB_BONUS

    return bonuses

def get_blended_projection(p, history_key):
    """
    Helper function to blend current projection with historical average.
    Returns blended projection or original projection if no history available.
    Increments historic_data_used counter when blending occurs.
    """
    global historic_data_used
    history_value = historic_averages.get(history_key)
    if history_value:
        historic_data_used += 1
        return AVERAGE_WEIGHT * p.proj + (1 - AVERAGE_WEIGHT) * history_value
    return p.proj

def calculate_adjusted_projection(p):
    """
    Calculate the adjusted projection for a player based on matchup factors.
    
    Base score is blended with historical averages, then adjusted by:
    - Spread advantage: How favorable the matchup is (Vegas spread)
    - Game script: Over/under to identify high/low scoring games
    - Home field advantage: Home vs away boost/penalty
    - Opponent strength: How good opponent defense is against this position
    - Team health: Injuries affecting the team or position
    
    Kickers are excluded from matchup weighting and use simple historical average blending.
    
    Returns a projection capped between MIN_SCORE and MAX_SCORE.
    """
    if not WEIGHTED:
        return p.proj

    # Kickers skip detailed weighting - just blend with historical average
    if p.pos == 'K':
        return get_blended_projection(p, name_map(p.name))

    # Low-salary non-defense players skip detailed weighting - just blend with historical average
    # Defenses always get matchup weighting regardless of salary
    if p.cost <= 4200 and p.pos != 'D':
        history_key = name_map(p.name)
        return get_blended_projection(p, history_key)

    # Step 1: Blend current projection with historical average
    # Historical data provides longer-term expectations
    history_key = name_map(p.name) if p.pos not in ['D', 'MVP'] else p.team.lower()
    base_score = get_blended_projection(p, history_key)

    # Step 2: Identify opponent for this matchup
    teams = p.matchup.split('@')
    opponent = teams[0] if p.team == teams[1] else teams[1]

    # Step 3: Calculate matchup adjustments
    matchup_bonus = 0

    # GAME TOTAL ADJUSTMENT: Over/under indicates high/low scoring game
    # Calculate this first since it affects spread weighting
    # High O/U = more total points in game = higher individual scoring
    # For defenses, high O/U is bad (more opponent points)
    point_bonus = z_map.get(p.team, 0)
    ou_bonus = calculate_overunder_bonus(p, point_bonus)
    matchup_bonus += ou_bonus

    # SPREAD ADJUSTMENT: Vegas spread indicates how favorable the matchup is
    # For OFFENSIVE players: positive spread = unfavored = fewer scoring opportunities
    # For DEFENSES: positive spread = facing strong offense = worse matchup
    spread_bonus = -favor_map.get(p.team, 0) / FAVOR_DIVISION
    
    # For defenses, invert the spread bonus (they benefit from facing bad offenses)
    if p.pos in ['D', 'MVP']:
        spread_bonus = -favor_map.get(p.team, 0) / FAVOR_DIVISION
        # In high-scoring games, even strong defenses give up points
        # So heavily dampen spread advantage in high O/U games
        spread_weight = 1.0 - (point_bonus * 0.5)  # Stronger dampening for defenses
    else:
        # In high-scoring games, spread matters less (everyone scores more)
        # In low-scoring games, spread matters more (must be on favored side)
        spread_weight = 1.0 - (point_bonus * 0.4)  # Moderate dampening for offensive players
    
    spread_bonus *= spread_weight
    matchup_bonus += spread_bonus

    # HOME FIELD ADVANTAGE: Home teams score more, away teams score less
    # This applies to all positions
    home_bonus = calculate_home_bonus(p)
    matchup_bonus += home_bonus

    # OPPONENT STRENGTH: How good is opponent defense?
    # (Uses rankings if available)
    ranking_bonus = calculate_ranking_bonus(p, opponent)
    matchup_bonus += ranking_bonus

    # TEAM HEALTH ADJUSTMENTS: How are injuries affecting this team/position?
    injury_bonus = calculate_injury_bonuses(p, opponent)
    matchup_bonus += injury_bonus

    # DEFENSIVE MATCHUP: For defenses and MVPs, account for opponent offense strength
    if p.pos in ['D', 'MVP']:
        matchup_bonus += excluded_bonus.get(opponent, 0) / 4

    # Step 4: Apply adjustments with safeguards
    # Only apply if player has minimum baseline or is a defense/MVP
    if p.pos in ['D', 'MVP'] or base_score >= MIN_SCORE:
        adjusted_proj = min(base_score + matchup_bonus, MAX_SCORE)
        # CRITICAL: Never project negative or too low below base score.
        adjusted_proj = max(adjusted_proj, base_score * .5)
        
        # For defenses, cap the upside boost to 2x of base score
        # Prevents massive overestimation for favored defenses
        if p.pos == 'D':
            adjusted_proj = min(adjusted_proj, base_score * 2)
        
        return adjusted_proj

    return base_score

# First, adjust all regular players
for p in players:
    p.average_score = m_score[p.pos if p.pos in m_score else p.pos.replace('MVP','D')]
    name = p.name.replace('.', '')

    # Calculate adjusted projection for regular players
    original_proj = p.proj
    p.proj = calculate_adjusted_projection(p)

    # Store the adjusted projection for MVP creation
    p.kv_store['adjusted_proj'] = p.proj

# For single game, create MVP slot with 1.5x salary and 1.5x ADJUSTED projection
if SINGLE_GAME:
    mvp_players = []
    regular_players = []

    for p in players:
        import copy
        mvp = copy.deepcopy(p)
        mvp.pos = 'MVP'
        mvp.cost = int(round(p.cost * 1.5))
        # Use the adjusted projection for the base, then apply MVP boost
        adjusted_base = p.kv_store.get('adjusted_proj', p.proj)
        mvp.proj = adjusted_base * 1.5
        mvp.name = p.name + ' (MVP)'
        mvp.kv_store['base_name'] = p.name  # Store original name
        mvp_players.append(mvp)
        mvp_base_names.add(p.name)

        # Keep regular players - CustomRule will prevent both versions from being selected
        regular_players.append(p)

    # For single game, include both regular and MVP versions
    # The optimizer constraint will enforce that only one version per player can be selected
    players.extend(mvp_players)

# Validate that all player teams are in spread data
teams_in_data = set(favor_map.keys())
missing_teams = set()

for p in players:
    if p.team not in teams_in_data:
        missing_teams.add(p.team)

if missing_teams:
    error_msg = "The following teams are missing from spread/matchup data:\n"
    for team in sorted(missing_teams):
        player_count = sum(1 for p in players if p.team == team)
        error_msg += f"  {team} ({player_count} players affected)\n"
    raise ValueError(error_msg)

# Now sort players into appropriate lists (including MVPs)
for p in players:
    # Sort players into appropriate lists
    base_fppg = float(p.kv_store.get('FPPG') or 0)
    
    # Extract opponent from matchup
    if p.matchup:
        teams = p.matchup.split('@')
        opponent = teams[0].strip() if p.team == teams[1].strip() else teams[1].strip()
    else:
        opponent = 'N/A'
    
    if p.pos == 'MVP':
        mvps.append((p.name, p.proj, p.cost, p.proj / p.cost, p.pos, base_fppg, opponent))
    elif p.pos == 'D':
        spread = favor_map.get(p.team, 0)
        defenses.append((p.team, p.proj, p.cost, p.proj / p.cost, base_fppg, spread, opponent))
    elif p.pos == 'QB' and p.cost >= MIN_QB_SALARY:
        # print('QB', p.name, p.cost, p.proj)
        point_bonus = z_map.get(p.team, 0)
        favor_bonus = -favor_map.get(p.team, 0) / FAVOR_DIVISION
        base_fppg = float(p.kv_store.get('FPPG', 0))
        qbs.append((name_map(p.name), p.proj, p.cost, p.proj / p.cost, point_bonus, favor_bonus, base_fppg, opponent))

print(f"players {len(players)}")
print(f"historic data used {historic_data_used} of {len(players)}")

# Build the starter map and filter MVPs using extracted functions
starter_map = build_starter_map(players, questionable_df)
filtered_mvps = filter_mvps(mvps, players, starter_map)

players 201
historic data used 0 of 201


In [397]:

# Best picks

print("\n" + "="*100)
print("SORTED DEFENSES")
print("="*100)
print(f"{'Team':<8} {'Proj':>8} {'Salary':>10} {'Base':>8} {'Spread':>8} {'Opp':>5} {'Value':>8}")
print("-"*100)
for team, proj, cost, value, base_fppg, spread, opp in sorted(defenses, key=lambda x: x[3], reverse=True):
    print(f"{team:<8} {proj:>8.2f} ${cost:>9,.0f} {base_fppg:>8.2f} {spread:>8.2f} {opp:>5} {(value*1000):>7.1f}x")

if SINGLE_GAME:
    print("\n" + "="*100)
    print("SORTED MVPs (Top 10 - Backups Shown Only if Starter Injured)")
    print("="*100)
    print(f"{'Name':<35} {'Proj':>8} {'Salary':>10} {'Base':>8} {'Value':>8} {'Games':>7} {'Opp':>5} {'Status':<10}")
    print("-"*100)
    
    # filtered_mvps and starter_map are already computed from the previous cell
    # Display the filtered MVPs
    for name, proj, cost, value, pos, base_fppg, games_played, opp, status in filtered_mvps[:10]:
        print(f"{name:<35} {proj:>8.2f} ${cost:>9,.0f} {base_fppg:>8.2f} {(value*1000):>7.1f}x {games_played:>7.0f} {opp:>5} {status:<10}")



SORTED DEFENSES
Team         Proj     Salary     Base   Spread   Opp    Value
----------------------------------------------------------------------------------------------------
PIT         15.47 $    4,400     7.73    -3.00   CLE     3.5x
CLE         13.87 $    4,100     6.93     3.00   PIT     3.4x
NO          12.80 $    4,200     6.67    -2.50   TEN     3.0x
SEA         14.25 $    4,700    10.67    -7.09   CAR     3.0x
NE          13.55 $    5,000     7.20   -13.50   NYJ     2.7x
TEN          8.19 $    3,600     5.73     2.50    NO     2.3x
BUF          9.01 $    4,000     6.67    -1.46   PHI     2.3x
LV           6.94 $    3,400     3.80     1.07   NYG     2.0x
PHI          8.54 $    4,300     7.87     1.46   BUF     2.0x
TB           8.06 $    4,500     5.93    -5.86   MIA     1.8x
NYG          6.58 $    3,700     3.73    -1.07    LV     1.8x
CAR          6.07 $    3,500     5.93     7.09   SEA     1.7x
LAR          8.24 $    4,800     8.13    -8.00   ATL     1.7x
JAC          8

In [398]:

print("\n" + "="*105)
print("SORTED QBs")
print("="*105)
print(f"{'Name':<35} {'Proj':>8} {'Salary':>10} {'Value':>8} {'O/U':>8} {'Spread':>8} {'Opp':>5} {'Base':>8}")
print("-"*105)
for name, proj, cost, value, ou_bonus, spread_bonus, base_fppg, opp in sorted(qbs, key=lambda x: x[-2], reverse=True):
    print(f"{name:<35} {proj:>8.2f} ${cost:>9,.0f} {(value*1000):>7.1f}x {ou_bonus:>8.2f} {spread_bonus:>8.2f} {opp:>5} {base_fppg:>8.2f}")


SORTED QBs
Name                                    Proj     Salary    Value      O/U   Spread   Opp     Base
---------------------------------------------------------------------------------------------------------
Josh Allen                             24.31 $    8,800     2.8x     0.00     0.36   PHI    24.03
Matthew Stafford                       24.51 $    8,400     2.9x     1.08     2.00   ATL    21.86
Drake Maye                             24.43 $    8,400     2.9x    -0.22     3.38   NYJ    20.95
Brock Purdy                            23.38 $    7,400     3.2x     1.56     0.76   CHI    20.90
Trevor Lawrence                        23.96 $    8,000     3.0x     0.79     1.56   IND    20.38
Jalen Hurts                            20.29 $    8,600     2.4x     0.00    -0.36   BUF    19.94
Caleb Williams                         20.21 $    7,700     2.6x     1.56    -0.76    SF    18.91
Jacoby Brissett                        19.09 $    7,600     2.5x     1.68    -1.76   CIN    17.86


In [399]:
# resets
LOCKED = []
BANNED = []
BLOCKED_TEAMS = []

player_settings = PlayerPoolSettings()

MIN_PROJ = 0

# Positive value means allow teams that are unfavored to win.
min_favored = 10
MIN_LIMIT = min_favored - 2

roster = None
best_roster = None
best_score = 0

get_score = lambda roster: sum([p.proj for p in roster.players])

def block_function(p):
    store = p.kv_store
    played = int(float(store.get('Played') or 0))
    if SINGLE_GAME:
        # require min number of games played
        return played < MIN_PLAYED
        # return False
    name = p.name if p.pos != 'D' else p.team

    if p.team in BLOCKED_TEAMS:
        return True

    if p.pos == 'D' and (p.cost > 5000):
        return True

    if p.pos == 'QB' and p.cost < MIN_QB_SALARY or p.cost > max_salary:
        return True

    # print(favor_map[name], p.__dict__, min_favored)
    cost_filter = p.pos != 'QB' and (p.cost > max_salary or played <1)# or played > WEEK+1)

    should_skip = (p.proj < MIN_PROJ and p.pos != 'D') or (p.proj < 10 and p.pos == 'QB')  or cost_filter

    #print(p.name, played, MAX_PLAYED)
    return should_skip

def print_optimized_roster(roster, min_favored_factor):
    """
    Print an optimized roster in table format with weighting factor info.
    Sorted by position order: QB, RBs, WRs, FLEX, TE, D
    
    Args:
        roster: The optimized NFL roster
        min_favored_factor: The current min_favored weighting factor
        iteration_num: Which iteration of the optimization loop this is
    """
    # Count unique teams in roster
    teams_in_roster = set(p.team for p in roster.players)
    num_teams_in_roster = len(teams_in_roster)
    max_teams_possible = len(set_teams)
    
    print("\n" + "="*120)
    print(f"OPTIMIZED LINEUP (weighting_factor={min_favored_factor}, total_score={get_score(roster):.2f}, teams={num_teams_in_roster}/{max_teams_possible})\n---")
    
    # Define position order
    position_order = {'QB': 0, 'RB': 1, 'WR': 2, 'TE': 3, 'D': 4, 'MVP': 5, 'FLEX': 6}
    
    # Create table data and sort by position
    roster_data = []
    total_salary = 0
    for i, p in enumerate(roster.players, 1):
        # Get base projection (original FPPG) and adjusted projection
        base_fppg = float(p.kv_store.get('FPPG', 0))
        adjusted_proj = p.proj
        salary = int(p.cost)  # Convert to int to avoid decimal point formatting issues
        
        # Extract opponent from matchup (format: "AwayTeam@HomeTeam")
        if p.matchup:
            teams = p.matchup.split('@')
            if len(teams) == 2:
                opponent = teams[0].strip() if p.team == teams[1].strip() else teams[1].strip()
            else:
                opponent = 'N/A'
        else:
            opponent = 'N/A'
        
        # Get spread from favor_map (already properly signed: + for favorites, - for underdogs)
        spread = favor_map.get(p.team, 0)
        
        total_salary += salary
        
        # Get position order for sorting (use the mapped value, default to 99 if not found)
        pos_order = position_order.get(p.pos, 99)
        
        roster_data.append({
            'Slot': f"{p.pos:5}",
            'Name': p.name[:25],
            'Team': p.team,
            'Opp': opponent,
            'Salary': f"${salary:,}",
            'Base Proj': f"{base_fppg:6.2f}",
            'Weighted Proj': f"{adjusted_proj:6.2f}",
            'Value': f"{(adjusted_proj/salary)*1000:6.1f}x",
            'Spread': f"{spread:+.1f}",
            'pos_order': pos_order
        })
    
    # Sort by position order
    roster_data.sort(key=lambda x: x['pos_order'])
    
    # Print table header
    print(f"{'Slot':<6} {'Name':<27} {'Team':<5} {'Opp':<5} {'Salary':<10} {'Base Proj':<12} {'Weighted Proj':<15} {'Value':<10} {'Spread':<8}")
    print("-"*120)
    
    # Print rows
    for row in roster_data:
        print(f"{row['Slot']:<6} {row['Name']:<27} {row['Team']:<5} {row['Opp']:<5} {row['Salary']:<10} {row['Base Proj']:<12} {row['Weighted Proj']:<15} {row['Value']:<10} {row['Spread']:<8}")
    
    # Print summary
    print("-"*120)
    total_salary_formatted = f"{total_salary:,}"
    print(f"{'TOTAL':<6} {'':<27} {'':<5} ${total_salary_formatted:<9} {'':<12} {get_score(roster):<15.2f}")
    print("-"*3)


In [400]:

def build_optimizer_settings(players, block_function):
    """
    Build optimizer settings with custom rules to enforce position constraints.
    
    Rules created:
    - Block players according to block_function
    - At most 1 version (regular or MVP) per player
    - At most 1 RB per team
    - At most 1 WR per team
    - At most 1 QB per team
    - At most 1 TE per team
    
    Args:
        players: List of Player objects
        block_function: Function to determine which players to exclude
    
    Returns:
        OptimizerSettings object configured with all custom rules
    """
    custom_rules = [
        CustomRule(
            group_a=lambda p: p,
            group_b=block_function,
            comparison=lambda sum, a, b: sum(b) == 0
        )
    ]

    # For single game: enforce that only ONE version (regular or MVP) of each player can be selected
    if SINGLE_GAME:
        player_versions = {}
        for p in players:
            base_name = p.name.replace(' (MVP)', '')
            if base_name not in player_versions:
                player_versions[base_name] = []
            player_versions[base_name].append(p)
        
        # Add constraint: at most 1 version per player
        for base_name, player_group in player_versions.items():
            if len(player_group) > 1:
                custom_rules.append(
                    CustomRule(
                        group_a=lambda p, base_name=base_name: p.name == base_name or p.name == f"{base_name} (MVP)",
                        group_b=lambda p: False,
                        comparison=lambda sum, a, b: sum(a) <= 1
                    )
                )

        return OptimizerSettings(
            custom_rules=custom_rules,
            min_teams=2
        )

    # Group RBs by team and enforce max 1 per team
    rb_teams = {}
    for p in players:
        if p.pos == 'RB':
            if p.team not in rb_teams:
                rb_teams[p.team] = []
            rb_teams[p.team].append(p)

    # Add constraint: at most 1 RB per team
    for team, team_rbs in rb_teams.items():
        if len(team_rbs) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'RB' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    # Group WRs by team and enforce max 1 per team
    wr_teams = {}
    for p in players:
        if p.pos == 'WR':
            if p.team not in wr_teams:
                wr_teams[p.team] = []
            wr_teams[p.team].append(p)

    # Add constraint: at most 1 WR per team
    for team, team_wrs in wr_teams.items():
        if len(team_wrs) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'WR' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    # Group QBs by team and enforce max 1 per team
    qb_teams = {}
    for p in players:
        if p.pos == 'QB':
            if p.team not in qb_teams:
                qb_teams[p.team] = []
            qb_teams[p.team].append(p)

    # Add constraint: at most 1 QB per team
    for team, team_qbs in qb_teams.items():
        if len(team_qbs) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'QB' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    # Group TEs by team and enforce max 1 per team
    te_teams = {}
    for p in players:
        if p.pos == 'TE':
            if p.team not in te_teams:
                te_teams[p.team] = []
            te_teams[p.team].append(p)

    # Add constraint: at most 1 TE per team
    for team, team_tes in te_teams.items():
        if len(team_tes) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'TE' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    return OptimizerSettings(
        custom_rules=custom_rules,
        min_teams=3
    )

In [401]:
BLOCKED_TEAMS = []

if SINGLE_GAME:
    LOCKED = ['Bo Nix (MVP)', 'Jaleel McLaughlin', 'Denver Broncos', 'Tyrone Tracy Jr.']
    BANNED = ['Harrison Butker (MVP)', "Aidan O'Connell", 'Jameis Winston', 'Jameis Winston (MVP)', 'RJ Harvey (MVP)', 'Isiah Pacheco', 'Travis Kelce', 'Kareem Hunt']
    # LOCKED = ['Brock Purdy (MVP)', 'RJ Harvey', 'Bo Nix (MVP)']
    # BANNED = ['Tyler Huntley', 'Courtland Sutton', 'Chris Oladokun', 'Jarrett Stidham', "Lil'Jordan Humphrey (MVP)", 'Jaleel McLaughlin',
    #            'Courtland Sutton (MVP)', 'Isiah Pacheco (MVP)', 'Tyler Badie', 'Xavier Worthy',' (MVP)Kansas City Chiefs', 'Ameer Abdullah', 'Jake Tonges', 'Jake Tonges (MVP)', 'Kendrick Bourne', 
    #           'Skyy Moore', 'JuJu Smith-Schuster',  'Marvin Mims Jr.', 'Troy Franklin (MVP)', 'Kansas City Chiefs', 'Luke Farrell', 'Jeremy McNichols', 'Kyle Williams', 'Patrick Taylor Jr.',
    #             'Mac Jones', 'RJ Harvey (MVP)', 'Mac Jones (MVP)', 'Philip Rivers']
    pass
else:
    LOCKED = [ 'Jake Ferguson', 'Baltimore Ravens', 'Parker Washington', 'Ashton Jeanty', 'Chase Brown', 'Jacory Croskey-Merritt',  'Bo Nix', 'Jahmyr Gibbs', 'Mark Andews']
    BANNED = ['Anthony Firkser', 'Hunter Henry', 'James Cook III', 'Kyren Williams', 'Christian Kirk', 'Jawhar Jordan',  "Tre' Harris", 'Matthew Golden', 'Jameis Winston', 'Isaiah Likely', 'Dontayvion Wicks', 'DeAndre Hopkins', 'Ladd McConkey','Zay Flowers',
               'Terry McLaurin', 'Jameson Williams', 'George Pickens', 'Kareem Hunt', 'Malik Davis', 'Marvin Mims Jr.', 'Kalif Raymond']
    #            'Justin Jefferson', 'Terry McLaurin',
    #           'Denver Broncos', 'Jahmyr Gibbs', 'Evan Engram',  'Washington Commanders', 'JuJu Smith-Schuster',
    #            'Isaac TeSlaa', 'Jalen Nailor', 'Zavier Scott', 'Jeremy McNichols', 'Detroit Lions']
    #           'KaVontae Turpin', 'Marvin Mims Jr.', 'Jalen Nailor', 'Justin Jefferson', 'Jeremy McNichols', 'Treylon Burks']
    # BANNED = ['Jalen Nailor', 'Treylon Burks', 'CeeDee Lamb', 'JuJu Smith-Schuster', 'Denver Broncos', 'Xavier Worthy']
    # BANNED = ['Washington Commanders', 'Zavier Scott', 'Tee Higgins', 'Khalil Shakir', 'DJ Moore',
    #            'Jacory Croskey-Merritt', 'Trevor Lawrence', 'Dalton Kincaid', 'Marvin Mims Jr.',
    #           'Jaxon Smith-Njigba', 'CeeDee Lamb', 'Darren Waller', 'Marvin Harrison Jr.', 'Jeremy McNichols',
    #           'JuJu Smith-Schuster',  'Zach Charbonnet', 'Brian Thomas Jr.', 'Hunter Henry',
    #           'Minnesota Vikings', 'George Pickens', 'KaVontae Turpin', 'Evan Engram',  'Trey McBride',
    #           'Xavier Worthy', 'James Cook III',  'T.J. Hockenson']
    pass

constraints = LineupConstraints(locked=LOCKED, banned=BANNED)


In [402]:

# Build optimizer settings with custom rules
opt_settings = build_optimizer_settings(players, block_function)

roster = run(
    rule_set=ACTIVE_RULE_SET,
    player_pool=players,
    verbose=False,
    optimizer_settings=opt_settings,
    constraints=constraints,
    player_settings=player_settings
)

if roster:
    # Print optimized roster
    print_optimized_roster(roster, min_favored)
    
    current_score = get_score(roster)
    if not best_score or current_score > best_score:
        best_score = current_score
        best_roster = roster
else:
    print("No solution")


OPTIMIZED LINEUP (weighting_factor=10, total_score=153.48, teams=8/22)
---
Slot   Name                        Team  Opp   Salary     Base Proj    Weighted Proj   Value      Spread  
------------------------------------------------------------------------------------------------------------------------
QB     Brock Purdy                 SF    CHI   $7,400      20.90        23.38             3.2x    -3.0    
RB     Christian McCaffrey         SF    CHI   $9,300      22.65        27.00             2.9x    -3.0    
RB     Chase Brown                 CIN   ARI   $8,000      13.97        18.36             2.3x    -7.0    
RB     Ashton Jeanty               LV    NYG   $7,500      13.63        15.32             2.0x    +1.1    
WR     Stefon Diggs                NE    NYJ   $6,500      10.33        16.18             2.5x    -13.5   
WR     Parker Washington           JAC   IND   $5,900       9.04        12.61             2.1x    -6.2    
WR     Emeka Egbuka                TB    MIA   $5,800 

In [403]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
if best_roster:
    roster = best_roster
    sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
    net_score = 0
    for p in sorted_players:
        advantage = favor_map[p.team]
        name = p.name.replace('.', '')
        print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj}  {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
        net_score += advantage
    print(f"---\nTotal adv: {net_score}\n")
    roster.players[-1].__dict__



Stefon Diggs                 -13.5                     16.18115514978702  1.1874461200420672
Chase Brown                  -7.04                     18.36053530120459  0
Parker Washington            -6.25                     12.612931289384116  0
Emeka Egbuka                 -5.86                     13.472299942320229  0
Christian McCaffrey          -3.02                     27  1.44
Brock Purdy                  -3.02                     23.375293672809626  1.44
Pittsburgh Steelers           -3.0                     15.466666666666667  0
Juwan Johnson                 -2.5                     11.689013705278768  2.5422545055042614
Ashton Jeanty                 1.07                     15.318851659733854  1.44
---
Total adv: -43.120000000000005



In [404]:
def calculate_diversity_score(roster):
    """
    Calculate a diversity score for a roster based on:
    1. Team diversification (penalize stacking)
    2. Position diversification (spread across positions)
    3. Correlation risk (players from same team tend to correlate)
    
    Higher score = more diverse roster (less correlation risk)
    Lower score = more stacked (higher correlation but higher ceiling)
    
    Args:
        roster: Optimized roster object with .players list
    
    Returns:
        dict with breakdown: {
            'overall_score': float (0-100),
            'team_diversity': float,
            'position_diversity': float,
            'stack_penalty': float,
            'team_breakdown': dict of team -> player count
        }
    """
    if not roster or not roster.players:
        return {'overall_score': 0, 'team_diversity': 0, 'position_diversity': 0}
    
    # Count players per team
    team_counts = defaultdict(int)
    position_counts = defaultdict(int)
    team_position_pairs = defaultdict(int)
    
    for p in roster.players:
        team_counts[p.team] += 1
        position_counts[p.pos] += 1
        team_position_pairs[(p.team, p.pos)] += 1
    
    roster_size = len(roster.players)
    num_teams = len(team_counts)
    num_positions = len(position_counts)
    
    # 1. TEAM DIVERSITY SCORE (0-40 points)
    # Ideal: 3+ teams, evenly distributed
    # Worst: 1 team (all players same team)
    ideal_teams = min(roster_size, 4)
    team_diversity = min((num_teams / ideal_teams) * 40, 40)
    
    # 2. POSITION DIVERSITY SCORE (0-30 points)
    # Penalize if too many of same position
    position_variance = np.var(list(position_counts.values())) if position_counts else 0
    max_position_variance = roster_size
    position_diversity = 30 * (1 - min(position_variance / max_position_variance, 1))
    
    # 3. STACK PENALTY (0-30 points)
    # Penalize when multiple players from same team at same position (correlation risk)
    stack_penalty_points = 0
    for (team, pos), count in team_position_pairs.items():
        if count > 1:
            stack_penalty_points += (count - 1) * 5
    
    # Penalize heavy team stacks (3+ players from same team)
    for team, count in team_counts.items():
        if count >= 3:
            stack_penalty_points += (count - 2) * 5
    
    stack_penalty = 30 - min(stack_penalty_points, 30)
    
    # OVERALL SCORE (0-100)
    overall_score = min(team_diversity + position_diversity + stack_penalty, 100)
    
    return {
        'overall_score': round(overall_score, 1),
        'team_diversity': round(team_diversity, 1),
        'position_diversity': round(position_diversity, 1),
        'stack_penalty': round(stack_penalty, 1),
        'team_breakdown': dict(team_counts),
        'position_breakdown': dict(position_counts),
        'num_teams': num_teams,
        'num_unique_positions': num_positions
    }

In [405]:

# Calculate and print diversity score for best roster
if best_roster and not SINGLE_GAME:
    diversity = calculate_diversity_score(best_roster)
    opt_score = get_score(best_roster)
    
    print("\nROSTER DIVERSITY ANALYSIS")
    print("-"*8)
    print(f"Overall Diversity Score: {diversity['overall_score']}/100")
    print(f"  - Team Diversity:      {diversity['team_diversity']}/40")
    print(f"  - Position Diversity:  {diversity['position_diversity']}/30")
    print(f"  - Stack Penalty:       {diversity['stack_penalty']}/30")
    print(f"\nTeams Used: {diversity['num_teams']} - {diversity['team_breakdown']}")
    print(f"Positions: {diversity['position_breakdown']}")
    
    # FLAG: Check if diversity is concerning despite high optimization score
    TARGET_DIVERSITY_SCORE = 70
    THRESHOLD_LOW_DIVERSITY = 50
    HIGH_OPT_THRESHOLD = 45  # Optimization score considered "high"
    
    if diversity['overall_score'] < THRESHOLD_LOW_DIVERSITY and opt_score >= HIGH_OPT_THRESHOLD:
        print("\n" + "⚠️ "*40)
        print("WARNING: HIGH CORRELATION RISK DETECTED")
        print("⚠️ "*40)
        print(f"Optimization Score: {opt_score:.2f} (good)")
        print(f"Diversity Score: {diversity['overall_score']}/100 (LOW - target {TARGET_DIVERSITY_SCORE}+)")
        print("\nThis lineup has high potential but HIGH VARIANCE due to:")
        
        if diversity['team_diversity'] < 20:
            most_common_team = max(diversity['team_breakdown'].items(), key=lambda x: x[1])
            print(f"  • Team concentration: {most_common_team[1]} players from {most_common_team[0]} (too concentrated)")
        
        if diversity['position_diversity'] < 15:
            most_common_pos = max(diversity['position_breakdown'].items(), key=lambda x: x[1])
            print(f"  • Position imbalance: {most_common_pos[1]} {most_common_pos[0]}s (unbalanced)")
        
        if diversity['stack_penalty'] < 15:
            print(f"  • Team stacking: Multiple players from same team in same position (correlation)")
        
        print(f"\nConsider adjusting constraints to improve diversity if you want:")
        print(f"  • Lower variance/safer lineups")
        print(f"  • Better hedge against team-specific outcomes")
        print(f"  • More balanced exposure")
        print("="*80 + "\n")
    
    elif diversity['overall_score'] < TARGET_DIVERSITY_SCORE:
        print(f"\n💡 Diversity score ({diversity['overall_score']}) could be improved (target: {TARGET_DIVERSITY_SCORE}+)\n")



ROSTER DIVERSITY ANALYSIS
--------
Overall Diversity Score: 96.8/100
  - Team Diversity:      40/40
  - Position Diversity:  26.8/30
  - Stack Penalty:       30/30

Teams Used: 8 - {'SF': 2, 'CIN': 1, 'LV': 1, 'NE': 1, 'JAC': 1, 'TB': 1, 'NO': 1, 'PIT': 1}
Positions: {'RB': 3, 'QB': 1, 'WR': 3, 'TE': 1, 'D': 1}


In [406]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open(UPLOAD_FILE, 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [407]:
output = pd.read_csv(UPLOAD_FILE)
output

,QB,RB,RB.1,WR,WR.1,WR.2,TE,FLEX,DEF
0,124752-85607:Brock Purdy,124752-55050:Christian McCaffrey,124752-89493:Chase Brown,124752-25079:Stefon Diggs,124752-137516:Parker Washington,124752-152655:Emeka Egbuka,124752-64808:Juwan Johnson,124752-167031:Ashton Jeanty,124752-12547:Pittsburgh Steelers


In [408]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

{'D': 6.324242424242424,
 'QB': 10.07846149588948,
 'RB': 9.72229000899764,
 'TE': 7.536632930650581,
 'WR': 7.714111230273399,
 'DEF': 6.324242424242424,
 'FLEX': 8.71820061963552}